In [1]:
# including the project directory to the notebook level
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import warnings

In [2]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    from keras import backend as K
    # from nn_source import models as mp
from dataprocess import dataprocessor as dp
import models as mp

Using TensorFlow backend.


In [3]:
"""This test checks that HyperOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import ray
from ray.tune import run
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
import argparse
from hyperopt import hp

In [4]:
# read the pickled file for ahu data
ahudata = dp.readfile('../data/processed/ahu1energy.pkl')

# return pickled df
ahu = ahudata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
ghidata = dp.readfile('../data/processed/ghi.pkl')

# return pickled df
ghi = ghidata.return_df(processmethods=['file2df'])

In [5]:
# selecting only required columns and rearranging them
ahu = ahu[[
    'AHU_1 outdoorAirTemp', 'WeatherDataProfile humidity',
    'AHU_1 supplyAirTemp', 'HW_BTU_METER currentKbtuDeltaReading',
    'CHW_BTU_METER currentKbtuDeltaReading'
]]

# renaming columns
ahu.columns = ['oat', 'orh', 'sat', 'hwe', 'cwe']

# Total energy is sum of heating and coling
ahu['totale'] = ahu['hwe'] + ahu['cwe']
# dropping heating and cooling energy columns
ahu = ahu.drop(columns=['hwe', 'cwe'])

In [6]:
# merging ahu and ghi data
df = dp.merge_df_columns([ahu, ghi])

# rearranging columns
df = df[['oat', 'orh', 'sat', 'Ghi', 'totale']]

In [7]:
df.head()

,oat,orh,sat,Ghi,totale
2018-07-12 13:55:00,89.554909,76.0,75.443275,872.0,35.590466
2018-07-12 14:00:00,89.863190,76.0,75.443275,866.0,33.027813
2018-07-12 14:05:00,89.863190,76.0,75.443275,859.0,31.769627
2018-07-12 14:10:00,90.484146,69.0,75.443275,848.0,34.195641
2018-07-12 14:15:00,90.826981,69.0,75.443275,836.0,31.082222


In [8]:
# Creating a list of 7 day dataframes for training
dflist = dp.df2dflist(df, subsequence=True, period=1, days=7, hours=0)

In [9]:
len(dflist)

22

In [10]:
# create list of training, testing arrays
weeklist = []
for weekdata in dflist:
    X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
        weekdata,
        predictorcols=['oat', 'orh', 'sat', 'Ghi'],
        outputcols=['totale'],
        scaling=True,
        feature_range=(0,1),
        reshaping=True,
        lag=-1
    )
    weeklist.append({
        'X_train':X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
    })

In [11]:
weeklist[0].keys()

dict_keys(['X_train', 'y_train', 'X_test', 'y_test'])

In [12]:
def objective_function(config, reporter):
    
    modeldesigndone = False
    
    for weekdata in config['weeklist']:
        
        X_train = weekdata['X_train']
        y_train = weekdata['y_train']
        X_test = weekdata['X_test']
        y_test = weekdata['y_test']
    
        if not modeldesigndone:
            #Instantiate learner model
            model = mp.lstm_model(inputdim=X_train.shape[-1], outputdim = y_train.shape[-1], period = 1)

            # Desing model architecture
            model.design_model(lstmhiddenlayers=[config['lstm_hidden_units']]*config['lstm_no_layers'], 
                           densehiddenlayers=[config['dense_hidden_units']]*config['dense_no_layers'], 
                       dropoutlist=[[], []], batchnormalizelist=[[], []])
            
            # creating early stopping and learning reate changing callbacks
            model.model_callbacks()
            
            modeldesigndone = True

        # train the model
        model.train_model(X_train, y_train, X_test, y_test, epochs = 200)

        # evaluate the model for metrics at this stage
        cvrmse, mae = model.evaluate_model(X_train, y_train, X_test, y_test)

        # reporter for tracking performance of individual process/ ray process
        # note if it is reported betwen weekly data loop, then training might stop at the middle of the loop
        reporter(test_cvrmse=sum(cvrmse[1])/len(cvrmse[1]))
    

In [13]:
ray.init()

2020-03-08 18:18:29,262	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 18:18:29,265	INFO resource_spec.py:212 -- Starting Ray with 36.43 GiB memory available for workers and up to 18.23 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 18:18:29,675	INFO services.py:1078 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '129.59.104.221',
 'redis_address': '129.59.104.221:19207',
 'object_store_address': '/tmp/ray/session_2020-03-08_18-18-29_261133_9535/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-08_18-18-29_261133_9535/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-03-08_18-18-29_261133_9535'}

In [14]:
space = {
    'lstm_hidden_units': hp.choice("lstm_hidden_units", [1, 2, 4, 8, 16, 32, 64]),
    'lstm_no_layers': hp.choice("lstm_no_layers", [1, 2, 4, 8]),
    'dense_hidden_units': hp.choice("dense_hidden_units", [1, 2, 4, 8, 16, 32, 64]),
    'dense_no_layers' : hp.choice("dense_no_layers", [1, 2, 4, 8]),
}

config = {
    "num_samples": 100,
    "verbose":1,
    "resources_per_trial":{"cpu": 0.2},
    "config":{'weeklist':weeklist}
}

algo = HyperOptSearch(
    space,
    max_concurrent=40,
    metric="test_cvrmse",
    mode="min")


scheduler = AsyncHyperBandScheduler(
    metric="test_cvrmse",
    mode="min")

In [15]:
analysis = run(objective_function, 
               search_alg=algo, 
               scheduler=scheduler, 
               **config)

Trial name,status,loc,lstm_no_layers,dense_no_layers,dense_hidden_units,lstm_hidden_units,total time (s),iter
objective_function_07bd21f6,TERMINATED,,4,2,4,16,23.5616,1
objective_function_0139f71e,TERMINATED,,1,8,16,32,62.901,4
objective_function_00826086,TERMINATED,,2,2,2,2,21.9591,1
objective_function_facb9b27,TERMINATED,,4,2,64,64,48.9337,1
objective_function_facb9b26,TERMINATED,,1,4,8,8,90.8904,4
objective_function_f4be16b5,TERMINATED,,8,1,16,16,80.817,1
objective_function_f4be16b4,TERMINATED,,2,2,64,4,93.9155,4
objective_function_ef441d32,TERMINATED,,4,2,1,64,55.9477,1
objective_function_eea3a334,TERMINATED,,2,8,16,2,26.714,1
objective_function_e92c4e38,TERMINATED,,1,2,16,2,174.548,22


2020-03-08 18:27:25,669	INFO tune.py:352 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


In [26]:
for i in space.keys():
    print("Best config for {} is".format(i), analysis.get_best_config(metric="test_cvrmse",mode='min')[i])
    
ray.shutdown()

Best config for lstm_hidden_units is 8
Best config for lstm_no_layers is 1
Best config for dense_hidden_units is 32
Best config for dense_no_layers is 4


In [1]:
%load_ext tensorboard

In [2]:
%tensorboard --logdir ~/ray_results/objective_function/

In [2]:
%tensorboard --logdir ~/ray_results/objective_function/